# Deep Learning Preprocessing v2 

__Author:__ Jack Friedman <br>
__Date:__ 11/6/2023 <br>
__Purpose:__ Playground for converting data into position image

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Preprocessing_v2 import *
from DataLoader import load_data

In [2]:
# Load data
[games_df, players_df, plays_df, tracking_df] = load_data()

loaded games df
shape: (136, 9)
-----
loaded players df
shape: (1683, 7)
-----
loaded plays df
shape: (12486, 35)
-----
loading tracking frames...
loaded tracking frames
shape: (12187398, 17)
returning 4 frames


In [3]:
tracking_df_clean = preprocess_all_df(plays_df, games_df, players_df, tracking_df)

cleaning plays_df
final plays data shape: (6840, 289)
-----

cleaning games_df
-----

cleaning players_df
-----

cleaning tracking_df
original tracking df shape: (12187398, 17)
unique play and game id combos: (6840, 2)
filtered df shape: (8458178, 17)
number of merge errors: 0
joined plays and tracking dataframes
original tracking shape: (8458178, 17)
merged data shape: (8458178, 304)
-------
joined players and tracking dataframes
original tracking shape: (8458178, 304)
merged data shape: (8458178, 306)
-------
joined games and tracking dataframes
original tracking shape: (8458178, 306)
merged data shape: (8458178, 307)
-------
Old df shape:(8458178, 346)
New df shape:(8458178, 346)
-----



In [4]:
# Check
for var in ['X_std', 'Y_std', 'Dir_std', 'O_std']:
    print(tracking_df_clean[var].describe())

count    8.458178e+06
mean     6.164130e+01
std      2.501609e+01
min     -1.770000e+00
25%      4.097000e+01
50%      5.877000e+01
75%      8.157000e+01
max      1.229600e+02
Name: X_std, dtype: float64
count    8.458178e+06
mean     2.683753e+01
std      8.461925e+00
min     -1.489000e+01
25%      2.189000e+01
50%      2.681333e+01
75%      3.177000e+01
max      5.972000e+01
Name: Y_std, dtype: float64
count    8.458178e+06
mean     1.523117e+02
std      1.023835e+02
min      0.000000e+00
25%      6.999000e+01
50%      1.370200e+02
75%      2.224200e+02
max      3.600000e+02
Name: Dir_std, dtype: float64
count    8.458178e+06
mean     1.682117e+02
std      9.996620e+01
min      0.000000e+00
25%      8.673000e+01
50%      1.502200e+02
75%      2.617200e+02
max      3.600000e+02
Name: O_std, dtype: float64


In [5]:
tracking_df_clean[(tracking_df_clean['X_std'] > 10) & (tracking_df_clean['X_std'] < 11)]['X_std'].unique()

array([10.09      , 10.5       , 10.94      , 10.22      , 10.56      ,
       10.93      , 10.31      , 10.86      , 10.91      , 10.29      ,
       10.6       , 10.9       , 10.88      , 10.82      , 10.76      ,
       10.71      , 10.67      , 10.63      , 10.58      , 10.53      ,
       10.44      , 10.06      , 10.26      , 10.61      , 10.57      ,
       10.27      , 10.02      , 10.16      , 10.43      , 10.87000275,
       10.44000244, 10.33000183, 10.23999786, 10.23000336, 10.54000092,
       10.93000031, 10.19      , 10.36      , 10.79      , 10.75      ,
       10.41      , 10.11      , 10.8       , 10.81      , 10.83      ,
       10.85      , 10.87      , 10.92      , 10.39      , 10.78      ,
       10.32      , 10.14      , 10.01      , 10.35      , 10.73      ,
       10.51      , 10.65      , 10.74      , 10.45      , 10.54      ,
       10.68      , 10.12      , 10.33      , 10.94      , 10.7       ,
       10.48      , 10.3       , 10.01      , 10.04      , 10.13

In [6]:
tracking_df_clean['event'].unique()

array([nan, 'run', 'first_contact', 'tackle', 'ball_snap', 'handoff',
       'man_in_motion', 'fumble', 'out_of_bounds', 'play_action',
       'touchdown', 'pass_forward', 'lateral', 'autoevent_passforward',
       'autoevent_passinterrupted', 'line_set', 'qb_slide', 'shift',
       'pass_outcome_caught', 'run_pass_option', 'qb_sack',
       'autoevent_ballsnap', 'snap_direct', 'pass_arrived',
       'fumble_defense_recovered', 'fumble_offense_recovered', 'safety',
       'penalty_flag', 'penalty_accepted'], dtype=object)

In [12]:
tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 80) & ((tracking_df_clean['event'] == 'run') | (tracking_df_clean['event'] == 'handoff'))]

,gameId,playId,nflId,frameId,time,s,a,dis,event,week,...,club_NYG,club_NYJ,club_PHI,club_PIT,club_SEA,club_SF,club_TB,club_TEN,club_WAS,club_football
5,2022090800,80,35472.0,6,2022-09-08 20:24:36.200000,1.19,2.80,0.11,run,1,...,0,0,0,0,0,0,0,0,0,0
35,2022090800,80,38577.0,6,2022-09-08 20:24:36.200000,3.72,3.93,0.36,run,1,...,0,0,0,0,0,0,0,0,0,0
65,2022090800,80,41239.0,6,2022-09-08 20:24:36.200000,1.63,4.04,0.17,run,1,...,0,0,0,0,0,0,0,0,0,0
95,2022090800,80,42392.0,6,2022-09-08 20:24:36.200000,2.75,1.08,0.29,run,1,...,0,0,0,0,0,0,0,0,0,0
125,2022090800,80,42489.0,6,2022-09-08 20:24:36.200000,6.13,5.47,0.61,run,1,...,0,0,0,0,0,0,0,0,0,0
155,2022090800,80,42816.0,6,2022-09-08 20:24:36.200000,4.46,2.94,0.44,run,1,...,0,0,0,0,0,0,0,0,0,0
185,2022090800,80,43294.0,6,2022-09-08 20:24:36.200000,1.39,1.24,0.14,run,1,...,0,0,0,0,0,0,0,0,0,0
215,2022090800,80,43298.0,6,2022-09-08 20:24:36.200000,3.77,5.07,0.35,run,1,...,0,0,0,0,0,0,0,0,0,0
245,2022090800,80,43335.0,6,2022-09-08 20:24:36.200000,1.75,3.47,0.16,run,1,...,0,0,0,0,0,0,0,0,0,0
275,2022090800,80,44875.0,6,2022-09-08 20:24:36.200000,1.11,1.81,0.11,run,1,...,0,0,0,0,0,0,0,0,0,0
